In [1]:
import os
import torch
import timm
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Install the timm library to access a variety of pre-trained models including ViT


# Imports

# Data loading
# Assuming you have mounted Google Drive and your train and test folders are available under '/content/drive/MyDrive/breast_cancer_data'
train_transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])

train_dataset = ImageFolder('C:\\Users\\tusha\\Downloads\\AID710\\BC2\\RSNA_Crop\\BC2', transform=train_transform)
test_dataset = ImageFolder('C:\\Users\\tusha\\Downloads\\AID710\\BC2\\RSNA_Crop\\test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model
model = timm.create_model('vit_base_patch16_224', pretrained=True)
num_classes = len(train_dataset.classes)
model.head = torch.nn.Linear(model.head.in_features, num_classes)

# Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Metrics storage
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

# Training and Testing Loop
for epoch in range(5):
    # Training
    model.train()
    train_loss = 0.0
    train_preds, train_targets = [], []
    for batch in train_loader:
        inputs, targets = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        train_targets.extend(targets.cpu().numpy())
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(accuracy_score(train_targets, train_preds))

    # Testing
    model.eval()
    test_loss = 0.0
    test_preds, test_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            test_preds.extend(outputs.argmax(dim=1).cpu().numpy())
            test_targets.extend(targets.cpu().numpy())
    test_losses.append(test_loss / len(test_loader))
    test_accuracies.append(accuracy_score(test_targets, test_preds))

    print(f"Epoch {epoch+1}: Train Loss: {train_losses[-1]:.3f}, Train Acc: {train_accuracies[-1]:.2f}, Test Loss: {test_losses[-1]:.3f}, Test Acc: {test_accuracies[-1]:.2f}")

# Plotting
fig, axs = plt.subplots(2, 1)
axs[0].plot(train_losses, label='Train Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_title('Loss vs Epochs')
axs[0].legend()

axs[1].plot(train_accuracies, label='Train Accuracy')
axs[1].plot(test_accuracies, label='Test Accuracy')
axs[1].set_title('Accuracy vs Epochs')
axs[1].legend()

plt.tight_layout()
plt.show()
